In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import calendar
import statsmodels.api as sm

plt.style.use('ggplot')  # Красивые графики
plt.rcParams['figure.figsize'] = (15, 5)  # Размер картинок

В данном задании проведем анализ реальных данных. Воспользуемся нетипичной интерпретацией геометрического распределения - игра бейсбол. В бейсболе геометрическое распределение полезно для анализа вероятности того, что отбивающий получит удар, прежде чем он получит три удара; здесь цель - добиться успеха за 3 испытания.   
Рассмотрим финальную серию чемпионата МЛБ-2020 между Лос Анджелес Доджерс и Тампа-Бэй Рэйс. Проанализируем статистику бэттеров первых 5 матчей, чтобы оценить какие бэттеры должны чаще выходить на поле в последнем матче серии и сравним с тем, как на самом деле это было. Почему принято решение выбрать именно такой подход? Всё из-за того, что чем выше процент отбивания у бэттера, тем выше вероятность того, что произойдет успех. Докажем это:  
Пусть вероятность того, что бэттер отобьёт удар равна p. Тогда рассчитаем вероятность успеха:  
$$P(X=0) + P(X=1) + P(X=2) = q^0p + q^1p + q^2p = p + (1-p)p + (1-p)^2 p = p + p - p^2 + p -2p^2 + p^3 = p(p^2-3p+3)$$  
Пройдемся по циклу и убедимся, что при увеличении p увеличивается и вероятность успеха.

In [2]:
for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    print(p*(p**2-3*p+3))

0.271
0.488
0.6569999999999999
0.784
0.875
0.9359999999999999
0.9730000000000002
0.9919999999999999
0.9989999999999999


Очевидно, что с ростом вероятности p растет и вероятность успеха.  
Рассмотрим статистические показатели, которые нам пригодятся:  
AB - At Bats = PA - BB - IBB - HBP - CI - SF - SH (На бите): Выходы на биту бэттера, за исключением уоков, ударов мячом, пожертвованных ударов, умышленных помех со стороны защиты или других препятствий.   
H - Hits (Хиты): общее количество хитов (1B, 2B, 3B и HR). Хит - удар, давший возможность выйти на базу. При этом защита не совершила ошибку.  
BA - Batting average = H / AB (он же AVG - средний коэффициент результативности отбивания): число хитов, деленное на число выходов на биту.

In [33]:
match_1 = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-1.csv',',',parse_dates=['Игрок'])
match_2 = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-2.csv',',',parse_dates=['Игрок'])
match_3 = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-3.csv',',',parse_dates=['Игрок'])
match_4 = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-4.csv',',',parse_dates=['Игрок'])
match_5 = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-5.csv',',',parse_dates=['Игрок'])

In [34]:
match_1

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,2,-,0.000
1,Arozarena R.,TAM,3,-,0.000
2,Barnes A.,LAD,4,-,0.000
3,Bellinger C.,LAD,4,1,0.250
4,Betts M.,LAD,4,2,0.500
5,Brosseau M.,TAM,1,1,1.000
6,Choi Ji-Man,TAM,-,-,0.000
7,Diaz Y.,TAM,4,1,0.250
8,Hernandez E.,LAD,2,1,0.500
9,Kiermaier K.,TAM,3,2,0.667


In [35]:
match_2

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,4,1,0.167
1,Arozarena R.,TAM,3,1,0.167
2,Barnes A.,LAD,1,-,0.000
3,Bellinger C.,LAD,3,-,0.143
4,Betts M.,LAD,3,-,0.286
5,Brosseau M.,TAM,2,-,0.333
6,Choi Ji-Man,TAM,3,1,0.333
7,Diaz Y.,TAM,1,1,0.400
8,Hernandez E.,LAD,1,-,0.333
9,Kiermaier K.,TAM,4,-,0.286


In [36]:
match_3

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,3,1,0.222
1,Arozarena R.,TAM,4,1,0.200
2,Barnes A.,LAD,3,1,0.125
3,Bellinger C.,LAD,4,1,0.182
4,Betts M.,LAD,5,2,0.333
5,Choi Ji-Man,TAM,4,-,0.143
6,Hernandez E.,LAD,1,-,0.250
7,Kiermaier K.,TAM,2,-,0.222
8,Lowe B.,TAM,4,-,0.154
9,Margot M.,TAM,3,1,0.400


In [37]:
match_4

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,4,1,0.231
1,Arozarena R.,TAM,4,3,0.357
2,Bellinger C.,LAD,4,-,0.133
3,Betts M.,LAD,5,-,0.235
4,Brosseau M.,TAM,2,1,0.400
5,Choi Ji-Man,TAM,-,-,0.143
6,Diaz Y.,TAM,3,-,0.250
7,Hernandez E.,LAD,4,1,0.250
8,Kiermaier K.,TAM,4,2,0.308
9,Lowe B.,TAM,4,1,0.176


In [38]:
match_5

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,4,-,0.176
1,Arozarena R.,TAM,4,1,0.333
2,Barnes A.,LAD,2,-,0.100
3,Bellinger C.,LAD,4,1,0.158
4,Betts M.,LAD,5,1,0.227
5,Brosseau M.,TAM,-,-,0.400
6,Choi Ji-Man,TAM,-,-,0.143
7,Diaz Y.,TAM,3,2,0.364
8,Hernandez E.,LAD,1,-,0.222
9,Kiermaier K.,TAM,3,2,0.375


In [41]:
itog = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Итого.csv',',',parse_dates=['Игрок'])
itog

,Игрок,Команда,AB,H,BA
0,Adames W.,TAM,17,3,"0,176470588235294"
1,Arozarena R.,TAM,18,6,"0,333333333333333"
2,Barnes A.,LAD,10,1,"0,1"
3,Bellinger C.,LAD,19,3,"0,157894736842105"
4,Betts M.,LAD,22,5,"0,227272727272727"
5,Brosseau M.,TAM,5,2,"0,4"
6,Choi Ji-Man,TAM,7,1,"0,142857142857143"
7,Diaz Y.,TAM,11,4,"0,363636363636364"
8,Hernandez E.,LAD,9,2,"0,222222222222222"
9,Kiermaier K.,TAM,16,6,"0,375"


А теперь попробуем спрогнозировать кто из бэттеров будет подходить к бите больше в своей команде, анализируя приведенные выше статистические данные по итогам первых 5 матчей финальной серии. Выведем данные в порядке убывания.

In [30]:
prognoz_TAM = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Итого_TAM.csv',',',parse_dates=['Игрок'])
prognoz_TAM

,Игрок,Команда,AB,H,BA,Комментарий
0,Margot M.,TAM,15,6,"0,4","Лучший BA, один из лучших AB"
1,Arozarena R.,TAM,18,6,"0,333333333333333",Один из лидеров по AB и отличный BA
2,Adames W.,TAM,17,3,"0,176470588235294",Один из лидеров по AB и средний BA
3,Lowe B.,TAM,21,3,"0,142857142857143","Лучший AB, но значимо хуже BA"
4,Kiermaier K.,TAM,16,6,"0,375",Очень хороший BA и неплохой AB
5,Wendle J.,TAM,15,2,"0,133333333333333",Примерно равные средние показатели
6,Meadows A.,TAM,13,2,"0,153846153846154",Примерно равные средние показатели
7,Diaz Y.,TAM,11,4,"0,363636363636364","Отличный BA, но все же низкий AB"
8,Zunino M.,TAM,13,0,0,"Приличный AB, но худший BA"
9,Choi Ji-Man,TAM,7,1,"0,142857142857143",Низкие AB и BA


А теперь посмотрим, сколько в итоге было подходов к бите у игроков TAM. Выведем данные в порядке убывания.

In [39]:
match_6_TAM = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-6_TAM.csv',',',parse_dates=['Игрок'])
match_6_TAM

,Игрок,Команда,AB,H,BA
0,Margot M.,TAM,4,0,0.316
1,Arozarena R.,TAM,4,2,0.364
2,Adames W.,TAM,4,0,0.143
3,Lowe B.,TAM,3,0,0.125
4,Kiermaier K.,TAM,3,1,0.368
5,Wendle J.,TAM,3,0,0.111
6,Meadows A.,TAM,3,1,0.188
7,Zunino M.,TAM,3,1,0.063
8,Choi Ji-Man,TAM,2,0,0.111
9,Diaz Y.,TAM,1,0,0.333


In [29]:
prognoz_LAD = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Итого_LAD.csv',',',parse_dates=['Игрок'])
prognoz_LAD

,Игрок,Команда,AB,H,BA,Комментарий
0,Turner J.,LAD,22,8,"0,363636363636364","Лучший AB, один из лучших BA"
1,Betts M.,LAD,22,5,"0,227272727272727","Лучший AB, высокий BA"
2,Seager C.,LAD,17,8,"0,470588235294118","Высокий AB, лучший BA"
3,Muncy M.,LAD,18,7,"0,388888888888889","Высокий AB, один из лучших BA"
4,Taylor C.,LAD,20,4,"0,2","Высокий AB, неплохой BA"
5,Смит У.,LAD,21,3,"0,142857142857143","Высокий AB, неплохой BA"
6,Bellinger C.,LAD,19,3,"0,157894736842105","Высокий AB, неплохой BA"
7,Pederson J.,LAD,10,4,"0,4","Средний AB, высокий BA"
8,Hernandez E.,LAD,9,2,"0,222222222222222",Средние показатели
9,Barnes A.,LAD,10,1,"0,1",Средние показатели


А теперь посмотрим, сколько в итоге было подходов к бите у игроков LAD. Выведем данные в порядке убывания.

In [40]:
match_6_LAD = pd.read_csv('/home/alexander/Рабочий стол/Курсовая/Матч-6_LAD.csv',',',parse_dates=['Игрок'])
match_6_LAD

,Игрок,Команда,AB,H,BA
0,Betts M.,LAD,4,2,0.269
1,Muncy M.,LAD,4,-,0.318
2,Turner J.,LAD,3,-,0.320
3,Seager C.,LAD,3,-,0.400
4,Taylor C.,LAD,3,1,0.217
5,Смит У.,LAD,3,1,0.167
6,Bellinger C.,LAD,3,-,0.136
7,Barnes A.,LAD,3,1,0.154
8,Pollock A.,LAD,2,-,0.167
9,Hernandez E.,LAD,1,-,0.200


Оценка того, кто больше из игроков будет подходить к бите в своей команде оказалось достаточно точной, несмотря на то, что данные были взяты лишь по 5 последним играм, что подтверждает важность этих показателей у игроков. Конечно же, по данным за такой короткий период корректировку могли внести такие ситуации как, например, травмы игроков, из-за которых они не смогли бы участвовать в 6 финальном матче или же участвовать в нём не в полной мере.  
Обычно в бейсболе статистику игроков оценивают по всему прошедшему сезону, что даст более высокую точность. 

Как можно заметить, рассмотренные статистические показатели у игроков команды LAD лучше, чем у игроков команды TAM. Отсюда неудивительно, что LAD выиграли не только последний матч у TAD со счетом 3:1, но и всю финальную серию со счетом 4:2 и стали чемпионами MLB-2020.

Ссылка на данные:
https://www.scoreboard.com/ru/baseball/usa/mlb-2020/